In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import qutip
import theano
import theano.tensor as T
import QubitNetwork

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
net = QubitNetwork.QubitNetwork(num_qubits=4,
                       interactions=('all', ['xx', 'zz']),
                       self_interactions=('all', ['z']),
                       system_qubits=[0, 1, 2])
# net.num_self_interactions + net.num_interactions
net.hs_factors

array([[[ 0.,  0.,  0., ..., -0., -0., -0.],
        [ 0., -0.,  0., ..., -0., -0., -0.],
        [ 0.,  0., -0., ..., -0., -0., -0.],
        ..., 
        [ 0.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., -0., -0., -0.],
        [ 0., -0.,  0., ..., -0., -0., -0.],
        [ 0.,  0., -0., ..., -0., -0., -0.],
        ..., 
        [ 0.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., -0., -0., -0.],
        [ 0., -0.,  0., ..., -0., -0., -0.],
        [ 0.,  0., -0., ..., -0., -0., -0.],
        ..., 
        [ 0.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ..., -0., -0., -0.],
        [ 0., -0.,  0., ..., -0., -0., -0.],
        [ 0.,  0., -0., ..., -0., -0., -0.],
       

In [11]:
# theano.config.optimizer = 'None'
QubitNetwork.sgd_optimization(learning_rate=1, batch_size=20)

Building the model...
The output file is available at train_model.png
Let's roll!
Epoch 0, gradient: [-0.00183574 -0.02573504  0.01360229  0.00380621 -0.01270479  0.01360229
  0.00687333 -0.01370464  0.01360229 -0.00350901 -0.00158698  0.01360229
  0.00060448  0.00416686  0.01360229  0.01360229  0.01360229  0.01360229]
gradient: [-0.00393851 -0.00558442 -0.00139619  0.01115102 -0.00574447 -0.00139619
  0.03399678 -0.04407256 -0.00139619 -0.0020187  -0.0015919  -0.00139619
  0.01685386 -0.00335733 -0.00139619 -0.00139619 -0.00139619 -0.00139619]
gradient: [-0.00761719 -0.01976035  0.00281509  0.00305864  0.00302463  0.00281509
 -0.02921696 -0.02070996  0.00281509  0.01214951  0.00566191  0.00281509
  0.01244348  0.00426611  0.00281509  0.00281509  0.00281509  0.00281509]
gradient: [ 0.00175857 -0.0107642   0.00212884 -0.03386892  0.00863442  0.00212884
 -0.00908608 -0.01497464  0.00212884  0.00246237  0.00135298  0.00212884
  0.00839674 -0.01294655  0.00212884  0.00212884  0.00212884  0

ValueError: cannot convert float NaN to integer
Apply node that caused the error: Expm(Reshape{2}.0)
Toposort index: 31
Inputs types: [TensorType(float64, matrix)]
Inputs shapes: [(32, 32)]
Inputs strides: [(256, 8)]
Inputs values: ['not shown']
Outputs clients: [[InplaceDimShuffle{0,1}(Expm.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/lk/anaconda3/envs/theano/lib/python3.5/site-packages/ipykernel/ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/lk/anaconda3/envs/theano/lib/python3.5/site-packages/ipykernel/zmqshell.py", line 501, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/lk/anaconda3/envs/theano/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2717, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/lk/anaconda3/envs/theano/lib/pyth